In [2]:
import requests

In [3]:
r = requests.get('https://api.github.com/user', auth=('user', 'pass'))

In [4]:
r.status_code

403

In [5]:
 r.headers['content-type']

'application/json; charset=utf-8'

In [6]:
 r.encoding

'utf-8'

In [7]:
r.text

'{"message":"Maximum number of login attempts exceeded. Please try again later.","documentation_url":"https://developer.github.com/v3"}'

In [8]:
r.json()

{'message': 'Maximum number of login attempts exceeded. Please try again later.',
 'documentation_url': 'https://developer.github.com/v3'}

In [9]:
def get_html(url):
   _html = ""
   resp = requests.get(url)
   if resp.status_code == 200:
      _html = resp.text
   return _html

In [10]:
from bs4 import BeautifulSoup
URL = "http://comic.naver.com/webtoon/list.nhn?titleId=20853&weekday=tue&page=1"
html = get_html(URL)
soup = BeautifulSoup(html, 'html.parser')

In [11]:
 l = soup.find_all("a")

In [12]:
print(len(l))

129


In [13]:
webtoon_area = soup.find("table",{"class": "viewList"})

In [14]:
webtoon_area = soup.find("table",
      {"class": "viewList"}
        ).find_all("td", {"class":"title"})

In [15]:
def parse_html(html):
	"""
	입력받은 마음의 소리 웹툰 페이지 html에서 마음의소리의 회차, 제목 url을 추출하여
	tuple로 만들고, 리스트에 갯수대로 저장하여 반환한다
	:param html: string
	:return: 마음의 소리 정보가 담긴 리스트
	"""
	webtoon_list = list()
	soup = BeautifulSoup(html, 'html.parser')
	webtoon_area = soup.find("table",
			{"class": "viewList"}
	        ).find_all("td", {"class":"title"})
	for webtoon_index in webtoon_area:
		info_soup = webtoon_index.find("a")
		_url = info_soup["href"]
		_text = info_soup.text.split(".")
		_title  = ""
		_num = _text[0]
		if len(_text) > 1:
			_title = _text[1]
			
		webtoon_list.append((_num, _title, _url, ))
	return webtoon_list

In [16]:
def insert_webtoon_info(simple_redis, infos):
   for info in infos:
      res = simple_redis.redis_hash_set("maso", info[0], info)